In [26]:
import numpy as np
import pandas as pd
#import stata file using pandas
raw_data = pd.read_stata('./HCMST_2017_public_data_v1.1_stata/HCMST_2017_draft_v1.1.dta')
#import data schema
data_schema = pd.read_csv('./cleaned_HCMST_2017_schema.csv', header=0, index_col=0)
#uses variables labels in data schema to create a survey glossary of terms
data_glossary = data_schema['variable_label']

## Analysis Goals

The following are the questions we are trying to answer with the data set:

1) Where are the top fives places couples met that resulted in relationships that lasted the longest?

2) Does household income affect the length of the relationship?

3) Does religiosity affect the length of the relationship?

4) Do political leanings affect the length of the relationship?

5) Do differences in educational level affect the length of the relationship?

6) Does age difference between partners affect the length of the relationship?

**During Data Cleaning, we should only collect features which help us answer these questions**


In [27]:
#show all rows, set max column width to 1000px to see the full text
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 1000)
data_schema = data_schema.reset_index()
data_schema

,variable_name,storage_type,display_format,value_label,variable_label
0,CaseID,int,%8.0g,NaN,Case ID
1,CASEID_NEW,long,%12.0g,NaN,Longitudnal CaseID
2,qflag,byte,%8.0g,QFLAG,DOV: Qualification Flag
3,weight1,double,%12.0g,NaN,Post-Stratification weight for Genpop (n=2994)
4,weight1_freqwt,float,%9.0g,NaN,"wgt to CPS adult pop, scaled down by 2994/3110=round(weight1*2436295 > 95/3110)"
5,weight2,double,%12.0g,NaN,Post-Stratification weight for LGB (n=551)
6,weight1a,double,%12.0g,NaN,Post-Stratification weight for total consented Genpop respondents (n=3110)
7,weight1a_freqwt,float,%9.0g,NaN,"weighted up to CPS adult population, =round(weight1a*243629595/3110)"
8,weight_combo,float,%9.0g,NaN,"weight that combines all LGB subjects weighted down, with gen pop"
9,weight_combo_freqwt,float,%9.0g,NaN,frequency weight version of weight_combo


In [28]:
#choose categories to keep
cat_to_keep = ['S1', 'Q21A_Year', 'Q21D_2_Year', 'Q21E_2_Year', 'Q21F_2_Year',
               'ppp20072', 'interracial_5cat', 'ppage', 
               'w6_q9', 'ppeducat', 'w6_q10', 'ppgender', 'ppincimp', 'ppethm', 'w6_q6b', 'partyid7', 'w6_q12']
cat_new_labels = ['Married', 'FirstMet_Year', 'Married_Year', 'BreakUp_Year', 'P_DeathYear',
                  'Religious_Attendance', 'Interracial_Couple', 'Age', 'P_Age',
                 'Education', 'P_Education', 'Gender', 'Household_Income', 'Ethnicity', 'P_Ethnicity', 'Politics', 'P_Politics']

In [29]:
#use for loop to collect more categories
for i in range(152,167):
    cat_to_keep.append(data_schema['variable_name'].iloc[i])
    cat_new_labels.append(data_schema['variable_label'].iloc[i])

In [30]:
#use for loop to collect more categories
for i in range(236,272):
    cat_to_keep.append(data_schema['variable_name'].iloc[i])
    cat = data_schema['variable_name'].iloc[i]
    cat_new_labels.append(cat[len('hcm2017q24_'):])

In [31]:
#clean cat labels
for c in range(len(cat_new_labels)):
    if 'Q14: ' in cat_new_labels[c]:
        cat_new_labels[c] = cat_new_labels[c].replace('Q14: ', '')

In [32]:
#new data set
data_set = raw_data[cat_to_keep].copy()
data_set.columns = cat_new_labels 

In [33]:
data_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3510 entries, 0 to 3509
Data columns (total 68 columns):
 #   Column                             Non-Null Count  Dtype   
---  ------                             --------------  -----   
 0   Married                            3510 non-null   category
 1   FirstMet_Year                      2856 non-null   category
 2   Married_Year                       132 non-null    category
 3   BreakUp_Year                       438 non-null    category
 4   P_DeathYear                        100 non-null    category
 5   Religious_Attendance               3394 non-null   category
 6   Interracial_Couple                 3365 non-null   category
 7   Age                                3510 non-null   category
 8   P_Age                              3374 non-null   float64 
 9   Education                          3510 non-null   category
 10  P_Education                        3394 non-null   category
 11  Gender                             3510 non

In [34]:
#save cat_to_keep and cat_new_labels as a dict
cat_dict = {cat_to_keep[i]: cat_new_labels[i] for i in range(len(cat_to_keep))}

In [35]:
#We know that columns concerning years and age must be dtype numeric, convert them with to_numeric()
#but first, drop every example where numerical features = "Refused"

num_features = ['FirstMet_Year','Married_Year','BreakUp_Year','P_DeathYear','Age', 'P_Age']

for f in num_features:
    data_set.drop(data_set[data_set[f] == "Refused"].index, inplace=True)

c:\users\grant\documents\learningstuff\datascienceprojects\date_marriage_hcmst_2017\venv\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [36]:
#check to see that "Refused" has been removed from these features
for f in num_features:
    print(set(data_set[f]))
    print('\n')
data_set.shape

{nan, '1980', '1951', '1956', '1976', '1999', '1987', '1957', '1994', '1974', '1939', '1991', '1995', '1953', '1946', '1973', '2014', '2001', '2002', '1979', '1996', '1970', '1998', '1969', '1978', '1950', '2010', '1972', '1966', '1964', '1989', '1985', '2008', '2016', '2005', '1952', '2013', '1965', '2006', '1971', '2011', '1992', '1958', '1948', '2004', '1988', '2017', '1982', '1968', '1960', '1997', '1983', '2012', '1984', '1962', '1963', '1981', '2007', '1986', '1993', '1967', '1949', '2015', '1955', '1990', '2009', '1959', '2000', '1954', '2003', '1961', '1977', '1975'}


{nan, '1980', '1976', '1999', '1987', '1974', '1991', '1995', '1973', '2001', '2002', '1979', '1996', '1978', '1998', '1969', '1970', '1950', '2010', '1977', '1972', '1966', '1964', '1985', '1989', '1952', '2008', '1965', '2011', '2013', '1948', '1988', '1982', '2004', '1968', '1960', '2017', '1997', '1983', '2012', '1984', '1962', '1963', '1981', '2007', '1986', '1993', '1967', '2015', '1990', '2009', '1959', '2

(3439, 68)

In [37]:
#"Refused" removed. Convert numerical features with to_numeric()
data_set[num_features] = data_set.loc[:,num_features].apply(lambda x: pd.to_numeric(x, errors='coerce'))
data_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3439 entries, 0 to 3509
Data columns (total 68 columns):
 #   Column                             Non-Null Count  Dtype   
---  ------                             --------------  -----   
 0   Married                            3439 non-null   category
 1   FirstMet_Year                      2820 non-null   float64 
 2   Married_Year                       122 non-null    float64 
 3   BreakUp_Year                       410 non-null    float64 
 4   P_DeathYear                        93 non-null     float64 
 5   Religious_Attendance               3323 non-null   category
 6   Interracial_Couple                 3297 non-null   category
 7   Age                                3439 non-null   int64   
 8   P_Age                              3311 non-null   float64 
 9   Education                          3439 non-null   category
 10  P_Education                        3323 non-null   category
 11  Gender                             3439 non

In [38]:
#throw away age outliers, they are likely errors in the data
data_set.drop(data_set[data_set['P_Age'] < 17].index, inplace=True)

In [39]:
print(set(data_set['P_Age']))

{nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, nan, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 89.0, 90.0, 91.0, 94.0, 95.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 33.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan

In [40]:
data_set[['FirstMet_Year','Married_Year']][data_set['Married_Year'].notna()]

,FirstMet_Year,Married_Year
22,NaN,2015.0
31,NaN,1989.0
62,NaN,1972.0
86,NaN,1965.0
122,NaN,1992.0
148,NaN,1968.0
153,NaN,1981.0
178,NaN,1988.0
188,NaN,1992.0
254,NaN,1991.0


In [41]:
data_set[['BreakUp_Year','P_DeathYear']][data_set['P_DeathYear'].notna()]

,BreakUp_Year,P_DeathYear
9,NaN,1983.0
62,NaN,2012.0
69,NaN,2007.0
81,NaN,2012.0
86,NaN,1988.0
121,NaN,1988.0
148,NaN,2003.0
153,NaN,2003.0
188,NaN,1996.0
248,NaN,1987.0


In [42]:
#Married couples do not report the first year at which they met. Let's combine them as 'First_Together'
data_set.insert(loc=1,column='First_Together',value=pd.Series(data_set['FirstMet_Year'].copy()))
ind_list = list(data_set[['FirstMet_Year','Married_Year']][data_set['Married_Year'].notna()].index)
for i in ind_list:
        data_set.at[i,'First_Together'] = data_set.at[i,'Married_Year']
    
# #Couples for which the partner died do not report the year they break up. Let's combine them as 'End_Year'
data_set.insert(loc=3,column='End_Year',value=pd.Series(data_set['BreakUp_Year'].copy()))
ind_list = list(data_set[['BreakUp_Year','P_DeathYear']][data_set['P_DeathYear'].notna()].index)
for i in ind_list:
        data_set.at[i,'End_Year'] = data_set.at[i,'P_DeathYear']

In [43]:
#check to see if 'First_Together' has successfully incorporated 'Married_Year' values
data_set[['First_Together','Married_Year']][data_set['Married_Year'].notna()]

,First_Together,Married_Year
22,2015.0,2015.0
31,1989.0,1989.0
62,1972.0,1972.0
86,1965.0,1965.0
122,1992.0,1992.0
148,1968.0,1968.0
153,1981.0,1981.0
178,1988.0,1988.0
188,1992.0,1992.0
254,1991.0,1991.0


In [44]:
#check to see if 'End_Year' has successfully incorporated 'P_DeathYear' values
data_set[['End_Year','P_DeathYear']][data_set['P_DeathYear'].notna()]

,End_Year,P_DeathYear
9,1983.0,1983.0
62,2012.0,2012.0
69,2007.0,2007.0
81,2012.0,2012.0
86,1988.0,1988.0
121,1988.0,1988.0
148,2003.0,2003.0
153,2003.0,2003.0
188,1996.0,1996.0
248,1987.0,1987.0


In [45]:
#Once the columns 'First_Together' and 'End_Year' are created, drop original columns, convert "Refused" to np.nan
#Filter data set such that we only preserve entries where 'First_Together' is not np.nan
data_set.drop(columns=['FirstMet_Year', 'Married_Year', 'BreakUp_Year', 'P_DeathYear'],inplace=True)
data_set.dropna(subset=['First_Together'],inplace=True)

In [46]:
data_set.shape

(2924, 66)

In [47]:
#create a column to figure out years together. Survey is taken in 2017, assume that N/A represents the couple is still dating
data_set.insert(loc=1, column='Years_Together', value=pd.Series(data_set['End_Year'].copy()))
ind_list = list(data_set[data_set['End_Year'].isna()].index)
for i in ind_list:
    data_set.at[i,'Years_Together'] = 2017
data_set['Years_Together'] = abs(data_set['Years_Together'] - data_set['First_Together'])


In [48]:
#create a column to figure out age difference between partners
new_col = abs(data_set['Age'] - data_set['P_Age'])
data_set.insert(loc=6, column='Age_Diff', value=list(new_col))

In [49]:
data_set

,Married,Years_Together,First_Together,End_Year,Religious_Attendance,Interracial_Couple,Age_Diff,Age,P_Age,Education,...,internet_game,internet_chat,internet_org,public,blind_date,vacation,single_serve_nonint,business_trip,work_neighbors,met_online
1,"Yes, I am Married",34.0,1983.0,NaN,Never,no,3.0,55,52.0,Bachelor's degree or higher,...,no,no,no,no,no,no,no,no,no,no
2,"Yes, I am Married",11.0,2006.0,NaN,Once or twice a month,no,2.0,47,45.0,Bachelor's degree or higher,...,no,no,no,no,no,no,no,no,no,yes
4,"Yes, I am Married",34.0,1983.0,NaN,Once a year or less,no,0.0,59,59.0,Bachelor's degree or higher,...,no,no,no,no,yes,no,no,no,no,no
5,"Yes, I am Married",36.0,1981.0,NaN,Once a week,no,1.0,59,60.0,High school,...,no,no,no,no,no,no,yes,no,no,no
6,"Yes, I am Married",51.0,1966.0,NaN,Once a week,no,1.0,66,67.0,High school,...,no,no,no,no,yes,no,no,no,no,no
7,"Yes, I am Married",50.0,1967.0,NaN,Never,no,0.0,65,65.0,Some college,...,no,no,no,yes,no,no,no,no,no,no
10,"Yes, I am Married",9.0,2008.0,NaN,Never,no,10.0,33,43.0,Bachelor's degree or higher,...,no,no,no,no,no,no,no,no,no,no
14,"Yes, I am Married",10.0,2007.0,NaN,Never,yes,2.0,37,35.0,Bachelor's degree or higher,...,no,no,no,no,no,no,no,no,no,no
15,"Yes, I am Married",15.0,2002.0,NaN,Once a year or less,no,4.0,38,42.0,Bachelor's degree or higher,...,no,no,no,no,no,no,no,no,no,no
16,"No, I am not Married",10.0,2007.0,NaN,A few times a year,no,3.0,34,37.0,Some college,...,no,no,no,no,no,no,no,no,no,no


In [50]:
#export cleaned dataset as cleaned_data.csv
data_set.to_csv('clean_data.csv', index=False)